In [ ]:
#Problem variables
cylinder_length = 50 #m
cylinder_diameter = 3 #m
water_reservoir_length = 100 #m

water_density = 1000 #kg/m^3
air_density = 1.2 #kg/m^3


In [ ]:
def get_cylinder_volume(cylinder_length, cylinder_diameter):
    cylinder_radius = cylinder_diameter / 2
    cylinder_straight_length = cylinder_length - cylinder_diameter
    return cylinder_straight_length * 3.14159 * cylinder_radius**2 + (4/3) * 3.14159 * cylinder_radius**3 #volume of cylinder + volume of two hemispheres

In [ ]:
water_volume = get_cylinder_volume(water_reservoir_length, cylinder_diameter)
water_mass = water_volume * water_density

air_volume = get_cylinder_volume(cylinder_length, cylinder_diameter) - water_volume
air_mass = air_volume * air_density

In [ ]:
def get_center_of_mass(water_mass, air_mass, cylinder_length, water_reservoir_length):
    water_center_of_mass = water_reservoir_length / 2
    air_center_of_mass = cylinder_length / 2
    total_mass = water_mass + air_mass
    return (water_center_of_mass * water_mass + air_center_of_mass * air_mass) / total_mass

In [ ]:
import math

## Determine flow rate given gravity gradient

gravity_gradient = 0 #m/s^2 per meter
max_gravity = 9.8 #m/s^2

# I need to find pressure at end given depth of water in the cylinder
def get_gravity_at_depth(depth, gravity_gradient, end_gravity):
    return end_gravity - gravity_gradient * depth

def get_discharge_velocity(water_height, gravity_gradient, end_gravity):
        return math.sqrt(2 * (max_gravity - (water_height**2 / 2) * gravity_gradient))

# volume flow rate
def get_volume_flow_rate(water_height, orifice_diameter, gravity_gradient, end_gravity):
    return get_discharge_velocity(water_height, gravity_gradient, end_gravity) * 3.14159 * (orifice_diameter / 2)**2

# get time to empty - must integrate over time since flow rate is not constant
def get_time_to_empty(water_height, orifice_diameter, gravity_gradient, end_gravity, water_volume):
    lost_volume = 0
    time = 0
    dt = 0.01
    while lost_volume < water_volume:
        lost_volume += get_volume_flow_rate(water_height, orifice_diameter, gravity_gradient, end_gravity) * dt
        time += dt
    return time
    
